In [84]:
import pandas as pd
import numpy as np
import tensorflow as tf

要predict的是each rally result

In [147]:
game_num = 2
path = 'set1_test.csv'
df_combined = pd.DataFrame()
for i in range(0, game_num):
    df = pd.read_csv(path, dtype={'No.': str})
    df.insert(0, 'Game', i)
    df.insert(8, 'Nothing', np.nan)
    df = df.drop(index=df[df['Action'] == 'OP'].index.tolist())
    for idx, row in df.iterrows():
        if(row['Attempts'] == 1):
            df.loc[idx, 'Action'] = df.loc[idx, 'Action'] + '_1'
        else:
            df.loc[idx, 'Action'] = df.loc[idx, 'Action'] + '_0'
        if(pd.isna(row['Errors']) and pd.isna(row['Score'])):
            df.loc[idx, 'Nothing'] = 1
    
    df = df.drop(columns='Attempts')
    df_combined = pd.concat([df_combined, df])

df_combined = df_combined.reset_index(drop=True)
print(len(df_combined))
df_combined.head()

480


,Game,Team,Rally,No.,Space,Action,Errors,Nothing,Score
0,0,IRI,1,2,Y,JS_0,NaN,1.0,NaN
1,0,POL,1,14,8,R_0,NaN,1.0,NaN
2,0,POL,1,19,2,G_0,NaN,1.0,NaN
3,0,POL,1,6,3,A_0,1.0,NaN,NaN
4,0,IRI,1,22,1,B_0,NaN,NaN,1.0


By game_dataframe 下去做可能會比較好一點

In [151]:
def process(df):
    '''
    one df represents one game
    split: space, action, player(?), error&score
    label: 發球方是否得分？
    '''
    col = df.columns
    space_col = [c for c in df.columns if 'Space' in c]
    action_col = [c for c in df.columns if 'Action' in c]
    result_col = ['Errors', 'Score', 'Nothing']
    others_col = [c for c in df.columns if c not in space_col and c not in action_col and c not in result_col and c != 'Game' and c != 'Rally']

    rally_set = []
    rally_space_set = []
    rally_action_set = []
    rally_result_set = []
    for _, df_rally in df.groupby(['Game', 'Rally']):   # each rally in one game
        # print(df_rally.index[0])
        curr_team = df_combined['Team'][df_rally.index[0]]
        shot_set = []
        shot_space_set = []
        shot_action_set = []
        shot_result_set = []
        
        atk_sequence = []
        atk_space_sequence = []
        atk_action_sequence = []
        atk_result = []
        
        for _, shot in df_rally.iterrows():
            if(shot['Team'] != curr_team):
                shot_set.append(atk_sequence)
                shot_space_set.append(atk_space_sequence)
                shot_action_set.append(atk_action_sequence)
                shot_result_set.append(atk_result)   # 最後一動的結果 -> predict object
                
                curr_team = shot['Team']

                atk_sequence = []
                atk_space_sequence = []
                atk_action_sequence = []

            atk_space_sequence.append([shot[space_col]])
            atk_action_sequence.append([shot[action_col]])
            atk_sequence.append([shot[others_col]])
            atk_result = [shot[result_col]]
        
        # the last shot
        shot_set.append(atk_sequence)
        shot_space_set.append(atk_space_sequence)
        shot_action_set.append(atk_action_sequence)
        shot_result_set.append(atk_result)

        # one rally has been finished
        rally_set.append(shot_set)
        rally_space_set.append(shot_space_set)
        rally_action_set.append(shot_action_set)
        rally_result_set.append(shot_result_set)

    return rally_set, rally_space_set, rally_action_set, rally_result_set

In [152]:
df_combined = df_combined.replace({'IRI': 0, 'POL': 1})
df_dummy = pd.get_dummies(df_combined)

In [154]:
rally_set, rally_space_set, rally_action_set, rally_result_set = process(df_dummy)

In [ ]:
# rally_set[0][0][0][3:6]  # (rally, shot_sequence, attack_sequence, feature)

---